In [1]:
import torch
from torch import nn
import sklearn
from docx import Document
from glob import glob
from pathlib import Path

In [ ]:
def init_dataset(path_to_data):
    
    rtf_data, docx_data = [], []
    data_path = Path(path_to_data)
    docx_files = glob(f"{data_path}/*.docx")
    rtf_files = glob(f"{data_path}/*.rtf")

    print(f"Найдено всего: {len(docx_files) + len(rtf_files)} резюме.")

    print("Загрузка *.docx")
    for file in docx_files:
        paragraphs_data = []
        for text in Document(file).paragraphs:
            paragraphs_data.append(text.text.strip())
        docx_data.append(paragraphs_data)
    print("Успешно загружено!")

    print("Загрузка *.rtf")
    for file in rtf_files:
        ...
    print("Успешно загружено!")

    return rtf_data, docx_data

rtf_data, docx_data = init_dataset("data/")

rtf_data, docx_data

Найдено всего: 6 резюме.
Загрузка *.docx


TypeError: 'Document' object is not iterable